In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.tools import tool
from IPython.display import display, Markdown
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost
from dotenv import load_dotenv
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime
from autogen_core import SingleThreadedAgentRuntime
load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

In [2]:
@dataclass
class Message:
    content: str

In [3]:
host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

In [4]:
serper = GoogleSerperAPIWrapper()
 
@tool
def langchain_serper(query: str) -> str:
    """Fetch relevant information from the web for a given query"""
    return serper.run(query)


autogen_serper = LangChainToolAdapter(langchain_serper)

In [5]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

In [6]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [7]:


if ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")


In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [ ]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are several pros of using AutoGen in your AI Agent project:

1. **Customizability**: AutoGen allows for the creation of highly customizable AI agents that can be tailored to specific project needs.

2. **Multi-Modal Operation**: AutoGen supports different operational modes, enabling agents to utilize combinations of Large Language Models (LLMs), human inputs, and various tools efficiently.

3. **Microsoft Backing**: Being backed by Microsoft, AutoGen benefits from extensive resources and support, making it a reliable choice with a strong ecosystem.

4. **Open-Source Framework**: As an open-source platform, AutoGen fosters collaboration and improvement through contributions from the development community.

5. **Tool Integration**: It supports an array of tools including code executors and function callers, which allows agents to perform complex tasks autonomously, enhancing automation capabilities.

6. **Efficiency & Scalability**: AutoGen enhances the efficiency of developing AI applications and scales well to accommodate growing project requirements.

7. **Seamless Azure Integration**: The framework integrates smoothly with Microsoft Azure AI services, allowing easy deployment and leveraging cloud capabilities.

8. **Enhanced Collaboration**: AutoGen excels in enabling multi-agent collaboration, which can shorten feedback loops and facilitate more effective teamwork among agents.

These attributes make AutoGen a compelling choice for developing AI agents in your project. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for your AI Agent project:

1. **Steep Learning Curve**: AutoGen can be complex, especially for developers not familiar with state machines or graph-based logic. This may slow down development and require additional training.

2. **Documentation Issues**: The documentation for AutoGen is often criticized as being difficult to read and lacking sufficient examples, which can hinder understanding and implementation.

3. **Limited Built-in Compliance**: AutoGen has restrictions regarding compliance features, which may be a concern for projects that require strict adherence to regulatory standards.

4. **Security Limitations**: There are potential security issues, including the absence of explicit features like data encryption or intellectual property control.

5. **High Costs**: The cost associated with using AutoGen can be significant, particularly regarding API usage and token limits, making it less accessible for budget-constrained projects.

6. **Incompatibility with Open Source Models**: AutoGen may not seamlessly integrate with open-source AI models, limiting flexibility and customization options.

7. **Best Suited for Research Environments**: It may be more appropriate for experimental applications rather than straightforward commercial deployments, which could limit its practicality for some projects.

These factors should be considered carefully when deciding whether to adopt AutoGen in your AI Agent project. 

TERMINATE



## Decision:

Based on the research provided by the team, the decision is to **use AutoGen** for the project.

### Rationale:
The benefits of AutoGen significantly outweigh the drawbacks for our AI Agent project. Its customizability and multi-modal operation are essential for tailoring agents to specific needs, while the backing by Microsoft ensures reliability and support. The open-source nature promotes collaboration, and the tool integration capabilities enhance automation, making it a strong choice for developing sophisticated AI applications.

Although there are concerns regarding the learning curve, documentation, and compliance, these can be managed through dedicated training and support resources. Furthermore, the advantages of efficiency, scalability, and seamless Azure integration position AutoGen well for the project's requirements. Therefore, despite some cons, the potential to innovate and create effective AI agents makes using AutoGen a compelling option.

TERMINATE

: 

In [ ]:
import asyncio
import signal
import os

def nuclear_shutdown():
    print("NUCLEAR SHUTDOWN INITIATED – killing everything...")
    
    # Method 1: Force-kill the gRPC servers (most important)
    workers = [worker] if ALL_IN_ONE_WORKER else [worker1, worker2, worker]
    for w in workers:
        if hasattr(w, "_server") and w._server:
            try:
                w._server.stop(grace=0)
                print(f"   → gRPC server forced stop")
            except:
                pass
    
    # Method 2: Kill the current event loop tasks (kills hanging coroutines)
    try:
        loop = asyncio.get_running_loop()
        for task in asyncio.all_tasks(loop):
            task.cancel()
        print("   → All async tasks cancelled")
    except:
        pass
    
    # Method 3: OS-level kill (Windows safe)
    print("   → Sending SIGTERM to self...")
    os.kill(os.getpid(), signal.SIGTERM)

# RUN THIS WHEN IT HANGS
nuclear_shutdown()

In [ ]:
await host.stop()